### Aspect Orientation Programming

* Normally a function (should) have one core responsibility
* In a business use case we have many different functions like that

#### Use case ---> An eCommerce Application

##### Important Functions

* search_products()
* place_order()
* get_order_list()
* login()

##### Additional Redundant Functions

* we will have some common (redundant) tasks across these functions
* example
    * open database connections
    * handle exceptions
    * measure performance
        * we may want to see how much time some functions take
            * search_products
            * get_order_list
    * handle authentication
        * eg.
            * few functions should work only if user is authenticated
                * place_order
                * get_order list


##### Problem #1 code scattering

* same side asepects need to be applied in multiple functions
* example
    * authentication is applied to
        * place_order
        * get_order_list
    * performance montior is applied to
        * search_products
        * get_order_list

    * exception handling is applied to
        * all most all functionality

* Same logic written again and again
* If the logic changes we need to change everywehere


##### Problem #2 code tangling

* each core function performs multiple side jobs apart from core logic
* example
    * get_order_list
        * core logic
            * search_orders
        * side jobs
            * verify authentication
            * handle exception
            * measure performance

* each function has multiple logic
    * makes code less redable
* when we add new concern it may break old logic.
            
## Let us see the problem in code.

##### Let us create an authenticator
* it is a singleton model
* every one shoudld use same instance of autneticator

In [11]:
class SimpleAuthenticator:
    
    def __init__(self):
        self._user=None # no user logged in

    def is_authenticated(self):
        return self._user is not None
    
    def login(self,user,password):
        users=dict(vivek='p@ss', sanjay='p@ss')
        if user in users:
            _pass=users[user]
            if _pass==password:
                self._user=user
                return True
            
        self._user=None
        raise Exception('Invalid Credentials')
    
    def logout(self):
        self._user=None

SimpleAuthenticator._instance=SimpleAuthenticator()


In [12]:
SimpleAuthenticator._instance.login('vivek','p@ss')

True

In [13]:
SimpleAuthenticator._instance.is_authenticated()

True

In [14]:
SimpleAuthenticator._instance.logout()

In [15]:
SimpleAuthenticator._instance.is_authenticated()

False

In [16]:
SimpleAuthenticator._instance.login('prabhat','p@ss')

Exception: Invalid Credentials

#### Let us now write our business functions

In [17]:
import time

def search_products():
    #performance monitoring
    start=time.time()
    #core business logic
    result=[]

    #exception handling logic
    try: 
        #core business logic
        print('searching products')
        
        for i in range(10):
            time.sleep(0.75)
            result.append(f'Product#{i+1}')
        print('searching products done')

    except Exception as e:
        print(f'error occured :{e}')
    #performance monitoring
    end=time.time()
    print(f'Total Time taken is {end-start}')
    #core business logic
    return result

def get_order_list():
    #performance monitoring
    start=time.time()
    #core business logic

    #authentication validation logic
    if not SimpleAuthenticator._instance.is_authenticated():
        raise Exception('User is not authenticated')

    result=[]

    #exception handling logic
    try: 
        #core business logic
        print('searching products')
        
        for i in range(10):
            time.sleep(0.5)
            result.append(f'Order#{i+1}')
        print('searching products done')
    except Exception as e:
        print(f'error occured :{e}')

    #performance monitoring
    end=time.time()
    print(f'Total Time taken is {end-start}')
    #core business logic
    return result       
    


In [18]:
search_products()

searching products
searching products done
Total Time taken is 7.504557132720947


['Product#1',
 'Product#2',
 'Product#3',
 'Product#4',
 'Product#5',
 'Product#6',
 'Product#7',
 'Product#8',
 'Product#9',
 'Product#10']

#### A generic Performance Monitoring Logic

* can we  write a generic function to monitor performance of any function?
* Simple Performance monitoring had following steps
    1. store sthe start timer
    2. let the business logic run
    3. store the end time
    4. print the log
    
* now we can pass the business logic as call back to performance monitoring 

In [19]:
def find_performance(fn):
    start=time.time()
    fn()
    end=time.time()
    print(f'Total Time taken is {end-start} seconds')

In [22]:
def place_order():
    print('placing order...')
    time.sleep(0.4)
    print('order placed')

In [23]:
#by default placing order will not give you your performance
place_order()

placing order...
order placed


### Measuring the Performance of a function

In [26]:
find_performance(place_order)

placing order...
order placed
Total Time taken is 0.40082502365112305 seconds


#### Problem

* user will not want to wrap their call to place_order in find_performance
    * what if we have optimized and don't want to find_performance anymore?
* user will like to call their function normally


#### Let us comibine call back and closure together

* here we will create a wrapper that will 
    * wrap the core logic and additional concern together
* this wrapper can be considered as a **enchanced version** of my core logic

* the wrapper can be defined using a inner function
* the outer function can help us simply the wrapping per function


#### Code walkthrough

1. performance function takes **target** as callback
    * this target is my core functionality for which we want to measure performance
2. We define a **inner** wrapper
    * wrapper had two key elements
        1. performance monitoring
            * Line 3,5,6
        2. core logic invoked on line #4
        
8. we are returning wrapper function
    * wrapper functionc can be considered as an enchanced (more featurefull version) of actual target
    * It is still performing the core job
    * but it has additional features of performance monitoring

In [30]:
def performance(target):
    def wrapper():
        start=time.time()
        target()
        end=time.time()
        print(f'Total Time taken by {target.__name__} is {end-start} seconds')

    return wrapper

#### Let us see how this thing works

In [31]:
#default place_order doesn't measure performance
place_order()

placing order...
order placed


In [33]:
### But we can create an enhanced version of place_order

place_order_p = performance(place_order)

### now we can use place_order_p instead of place_order whereever we need

place_order_p()

### place_order_p can still place the order. but can also give you time taken


placing order...
order placed
Total Time taken by place_order is 0.4007441997528076 seconds


#### What if we always want to measure performance for place_order

* we may have used place_order() had many places
* now we may have to replace place_order() with place_order_p() everywhere
* this is undeseriable

##### We can replace the reference to the core function with wrapped_logic

In [36]:
def long_running_task():
    print('long running task started')
    time.sleep(2.5)
    print('long running task finished')

#replace original function with wrapped function
long_running_task = performance(long_running_task)

#now when user calls my core function, they actually call this wrapped function


In [37]:
long_running_task()

long running task started
long running task finished
Total Time taken by long_running_task is 2.5009372234344482 seconds


#### Important Note

* here long_running_task is not actually the long_running_task
* it is the wrapper function that **performance()** returned.

##### Can you provde that that long_running_task is not actually long_running_task

* just check the function name

In [38]:
long_running_task.__name__

'wrapper'